In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5707,2020-11-21T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1568,242,1810,34431,...,1485,1838,14489,1141,32148.0,19723.0,51871,875889,602679.0,NaN
5708,2020-11-21T17:00:00,ITA,9,Toscana,43.769231,11.255889,1798,296,2094,51046,...,-729,1892,37471,2165,72642.0,20134.0,92776,1440310,904959.0,NaN
5709,2020-11-21T17:00:00,ITA,10,Umbria,43.106758,12.388247,370,75,445,10978,...,55,408,9623,323,5995.0,15374.0,21369,385737,213635.0,NaN
5710,2020-11-21T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,149,11,160,1841,...,-79,91,3641,281,5268.0,655.0,5923,53960,31251.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5707,2020-11-21T17:00:00,19,Sicilia,1568,242,1810,34431,36241,1485,1838,14489,1141,32148.0,19723.0,51871,875889,602679.0
5708,2020-11-21T17:00:00,9,Toscana,1798,296,2094,51046,53140,-729,1892,37471,2165,72642.0,20134.0,92776,1440310,904959.0
5709,2020-11-21T17:00:00,10,Umbria,370,75,445,10978,11423,55,408,9623,323,5995.0,15374.0,21369,385737,213635.0
5710,2020-11-21T17:00:00,2,Valle d'Aosta,149,11,160,1841,2001,-79,91,3641,281,5268.0,655.0,5923,53960,31251.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-21', '2020-11-20')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-21T17:00:00,1,Piemonte,5150,390,5540,70552,76092,-453,2896,65346,5496,63328.0,83606.0,146934,1405087,890725.0
1,2020-11-21T17:00:00,2,Valle d'Aosta,149,11,160,1841,2001,-79,91,3641,281,5268.0,655.0,5923,53960,31251.0
2,2020-11-21T17:00:00,3,Lombardia,8314,936,9250,153435,162685,3044,8853,183472,20359,272904.0,93612.0,366516,3769051,2236696.0
3,2020-11-21T17:00:00,5,Veneto,2200,275,2475,68272,70747,2060,3567,45789,3190,29844.0,89882.0,119726,2658845,1022442.0
4,2020-11-21T17:00:00,6,Friuli Venezia Giulia,509,55,564,12193,12757,582,1043,10593,633,20317.0,3666.0,23983,658272,263440.0
5,2020-11-21T17:00:00,7,Liguria,1241,119,1360,14513,15873,-81,761,29416,2227,32708.0,14808.0,47516,560338,277654.0
6,2020-11-21T17:00:00,8,Emilia-Romagna,2594,241,2835,62245,65080,2146,2723,32672,5312,65480.0,37584.0,103064,1977482,1019829.0
7,2020-11-21T17:00:00,9,Toscana,1798,296,2094,51046,53140,-729,1892,37471,2165,72642.0,20134.0,92776,1440310,904959.0
8,2020-11-21T17:00:00,10,Umbria,370,75,445,10978,11423,55,408,9623,323,5995.0,15374.0,21369,385737,213635.0
9,2020-11-21T17:00:00,11,Marche,544,86,630,14492,15122,145,452,9775,1168,20054.0,6011.0,26065,387818,225892.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-20T17:00:00,1,Piemonte,5225,393,5618,70927,76545,-692,3861,62074,5419,62133.0,81905.0,144038,1386198,875229.0
1,2020-11-20T17:00:00,2,Valle d'Aosta,148,10,158,1922,2080,45,155,3474,278,5186.0,646.0,5832,53347,30994.0
2,2020-11-20T17:00:00,3,Lombardia,8304,930,9234,150407,159641,4061,9221,177832,20190,267227.0,90436.0,357663,3724757,2221955.0
3,2020-11-20T17:00:00,5,Veneto,2170,267,2437,66250,68687,2105,3468,44349,3123,29424.0,86735.0,116159,2640734,1016938.0
4,2020-11-20T17:00:00,6,Friuli Venezia Giulia,505,54,559,11616,12175,530,1018,10157,608,19425.0,3515.0,22940,651014,261532.0
5,2020-11-20T17:00:00,7,Liguria,1290,120,1410,14544,15954,-1073,761,28594,2207,32198.0,14557.0,46755,553862,275463.0
6,2020-11-20T17:00:00,8,Emilia-Romagna,2534,244,2778,60156,62934,1925,2533,32145,5265,64026.0,36318.0,100344,1957785,1011858.0
7,2020-11-20T17:00:00,9,Toscana,1792,295,2087,51782,53869,-255,2207,34894,2121,71442.0,19442.0,90884,1421783,897193.0
8,2020-11-20T17:00:00,10,Umbria,363,71,434,10934,11368,-17,394,9279,314,5879.0,15082.0,20961,380989,211832.0
9,2020-11-20T17:00:00,11,Marche,542,81,623,14354,14977,121,512,9475,1161,19698.0,5915.0,25613,384786,224325.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-21T17:00:00,1,Piemonte,5150,390,5540,70552,76092,-453,2896,65346,5496,63328.0,83606.0,146934,1405087,890725.0
1,2020-11-21T17:00:00,2,Valle d'Aosta,149,11,160,1841,2001,-79,91,3641,281,5268.0,655.0,5923,53960,31251.0
2,2020-11-21T17:00:00,3,Lombardia,8314,936,9250,153435,162685,3044,8853,183472,20359,272904.0,93612.0,366516,3769051,2236696.0
3,2020-11-21T17:00:00,4,Trentino-Alto Adige,863,79,942,13435,14377,-138,760,19222,1039,27917.0,6721.0,34638,640520,271022.0
4,2020-11-21T17:00:00,5,Veneto,2200,275,2475,68272,70747,2060,3567,45789,3190,29844.0,89882.0,119726,2658845,1022442.0
5,2020-11-21T17:00:00,6,Friuli Venezia Giulia,509,55,564,12193,12757,582,1043,10593,633,20317.0,3666.0,23983,658272,263440.0
6,2020-11-21T17:00:00,7,Liguria,1241,119,1360,14513,15873,-81,761,29416,2227,32708.0,14808.0,47516,560338,277654.0
7,2020-11-21T17:00:00,8,Emilia-Romagna,2594,241,2835,62245,65080,2146,2723,32672,5312,65480.0,37584.0,103064,1977482,1019829.0
8,2020-11-21T17:00:00,9,Toscana,1798,296,2094,51046,53140,-729,1892,37471,2165,72642.0,20134.0,92776,1440310,904959.0
9,2020-11-21T17:00:00,10,Umbria,370,75,445,10978,11423,55,408,9623,323,5995.0,15374.0,21369,385737,213635.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-20T17:00:00,1,Piemonte,5225,393,5618,70927,76545,-692,3861,62074,5419,62133.0,81905.0,144038,1386198,875229.0
1,1,2020-11-20T17:00:00,2,Valle d'Aosta,148,10,158,1922,2080,45,155,3474,278,5186.0,646.0,5832,53347,30994.0
2,2,2020-11-20T17:00:00,3,Lombardia,8304,930,9234,150407,159641,4061,9221,177832,20190,267227.0,90436.0,357663,3724757,2221955.0
3,19,2020-11-20T17:00:00,4,Trentino-Alto Adige,885,76,961,13554,14515,325,970,18354,1009,27295.0,6583.0,33878,632736,268359.0
4,3,2020-11-20T17:00:00,5,Veneto,2170,267,2437,66250,68687,2105,3468,44349,3123,29424.0,86735.0,116159,2640734,1016938.0
5,4,2020-11-20T17:00:00,6,Friuli Venezia Giulia,505,54,559,11616,12175,530,1018,10157,608,19425.0,3515.0,22940,651014,261532.0
6,5,2020-11-20T17:00:00,7,Liguria,1290,120,1410,14544,15954,-1073,761,28594,2207,32198.0,14557.0,46755,553862,275463.0
7,6,2020-11-20T17:00:00,8,Emilia-Romagna,2534,244,2778,60156,62934,1925,2533,32145,5265,64026.0,36318.0,100344,1957785,1011858.0
8,7,2020-11-20T17:00:00,9,Toscana,1792,295,2087,51782,53869,-255,2207,34894,2121,71442.0,19442.0,90884,1421783,897193.0
9,8,2020-11-20T17:00:00,10,Umbria,363,71,434,10934,11368,-17,394,9279,314,5879.0,15082.0,20961,380989,211832.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-21T17:00:00,1,Piemonte,5150,390,5540,70552,76092,-453,2896,65346,5496,63328.0,83606.0,146934,1405087,890725.0,4341375
1,2020-11-21T17:00:00,2,Valle d'Aosta,149,11,160,1841,2001,-79,91,3641,281,5268.0,655.0,5923,53960,31251.0,125501
2,2020-11-21T17:00:00,3,Lombardia,8314,936,9250,153435,162685,3044,8853,183472,20359,272904.0,93612.0,366516,3769051,2236696.0,10103969
3,2020-11-21T17:00:00,4,Trentino-Alto Adige,863,79,942,13435,14377,-138,760,19222,1039,27917.0,6721.0,34638,640520,271022.0,1074819
4,2020-11-21T17:00:00,5,Veneto,2200,275,2475,68272,70747,2060,3567,45789,3190,29844.0,89882.0,119726,2658845,1022442.0,4907704
5,2020-11-21T17:00:00,6,Friuli Venezia Giulia,509,55,564,12193,12757,582,1043,10593,633,20317.0,3666.0,23983,658272,263440.0,1211357
6,2020-11-21T17:00:00,7,Liguria,1241,119,1360,14513,15873,-81,761,29416,2227,32708.0,14808.0,47516,560338,277654.0,1543127
7,2020-11-21T17:00:00,8,Emilia-Romagna,2594,241,2835,62245,65080,2146,2723,32672,5312,65480.0,37584.0,103064,1977482,1019829.0,4467118
8,2020-11-21T17:00:00,9,Toscana,1798,296,2094,51046,53140,-729,1892,37471,2165,72642.0,20134.0,92776,1440310,904959.0,3722729
9,2020-11-21T17:00:00,10,Umbria,370,75,445,10978,11423,55,408,9623,323,5995.0,15374.0,21369,385737,213635.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-21T17:00:00,1,Piemonte,5150,390,5540,70552,76092,-453,2896,65346,5496,63328.0,83606.0,146934,1405087,890725.0,4341375,575
1,2020-11-21T17:00:00,2,Valle d'Aosta,149,11,160,1841,2001,-79,91,3641,281,5268.0,655.0,5923,53960,31251.0,125501,20
2,2020-11-21T17:00:00,3,Lombardia,8314,936,9250,153435,162685,3044,8853,183472,20359,272904.0,93612.0,366516,3769051,2236696.0,10103969,1036
3,2020-11-21T17:00:00,4,Trentino-Alto Adige,863,79,942,13435,14377,-138,760,19222,1039,27917.0,6721.0,34638,640520,271022.0,1074819,106
4,2020-11-21T17:00:00,5,Veneto,2200,275,2475,68272,70747,2060,3567,45789,3190,29844.0,89882.0,119726,2658845,1022442.0,4907704,1016
5,2020-11-21T17:00:00,6,Friuli Venezia Giulia,509,55,564,12193,12757,582,1043,10593,633,20317.0,3666.0,23983,658272,263440.0,1211357,180
6,2020-11-21T17:00:00,7,Liguria,1241,119,1360,14513,15873,-81,761,29416,2227,32708.0,14808.0,47516,560338,277654.0,1543127,209
7,2020-11-21T17:00:00,8,Emilia-Romagna,2594,241,2835,62245,65080,2146,2723,32672,5312,65480.0,37584.0,103064,1977482,1019829.0,4467118,634
8,2020-11-21T17:00:00,9,Toscana,1798,296,2094,51046,53140,-729,1892,37471,2165,72642.0,20134.0,92776,1440310,904959.0,3722729,523
9,2020-11-21T17:00:00,10,Umbria,370,75,445,10978,11423,55,408,9623,323,5995.0,15374.0,21369,385737,213635.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-21T17:00:00,3,Lombardia,8314,936,9250,153435,162685,3044,8853,...,90.35,3028,-368,44294,14741.0,169,5640,19.99,0.08762,3.63
1,2020-11-21T17:00:00,5,Veneto,2200,275,2475,68272,70747,2060,3567,...,27.07,2022,99,18111,5504.0,67,1440,19.70,0.07268,2.44
2,2020-11-21T17:00:00,15,Campania,2260,198,2458,98677,101135,2143,3554,...,39.21,2130,-672,24692,12294.0,32,1379,14.39,0.06143,2.30
3,2020-11-21T17:00:00,1,Piemonte,5150,390,5540,70552,76092,-453,2896,...,67.83,-375,-965,18889,15496.0,77,3272,15.33,0.06671,3.38
4,2020-11-21T17:00:00,8,Emilia-Romagna,2594,241,2835,62245,65080,2146,2723,...,38.01,2089,190,19697,7971.0,47,527,13.82,0.06096,2.31
5,2020-11-21T17:00:00,12,Lazio,3257,337,3594,76282,79876,2130,2658,...,39.79,2075,-9,30046,25196.0,37,491,8.85,0.04532,1.71
6,2020-11-21T17:00:00,9,Toscana,1798,296,2094,51046,53140,-729,1892,...,56.60,-736,-315,18527,7766.0,44,2577,10.21,0.05082,2.49
7,2020-11-21T17:00:00,19,Sicilia,1568,242,1810,34431,36241,1485,1838,...,41.16,1454,204,9386,6114.0,43,310,19.58,0.03699,1.04
8,2020-11-21T17:00:00,16,Puglia,1432,197,1629,28791,30420,986,1377,...,53.39,967,-79,9803,5414.0,19,372,14.05,0.03435,1.05
9,2020-11-21T17:00:00,6,Friuli Venezia Giulia,509,55,564,12193,12757,582,1043,...,30.56,577,25,7258,1908.0,25,436,14.37,0.08610,1.98


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,936,100,1036,90.35
1,Trentino-Alto Adige,79,27,106,74.53
2,Piemonte,390,185,575,67.83
3,Umbria,75,42,117,64.10
4,Marche,86,57,143,60.14
5,Liguria,119,90,209,56.94
6,Toscana,296,227,523,56.60
7,Valle d'Aosta,11,9,20,55.00
8,Puglia,197,172,369,53.39
9,Abruzzo,73,75,148,49.32


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,2020-11-19T17:00:00,22,P.A. Trento,407,39,446,2540,2986,126,266,9985,556,7096.0,6431.0,13527,340985,125144.0
540,2020-11-20T17:00:00,21,P.A. Bolzano,468,38,506,11005,11511,307,736,8160,446,20117.0,0.0,20117,288707,142658.0
541,2020-11-20T17:00:00,22,P.A. Trento,417,38,455,2549,3004,18,234,10194,563,7178.0,6583.0,13761,344029,125701.0
542,2020-11-21T17:00:00,21,P.A. Bolzano,450,39,489,11186,11675,164,548,8528,462,20665.0,0.0,20665,291617,143675.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2020-11-17T17:00:00,4,Trentino-Alto Adige,870,79,949,12746,13695,289,534,...,12.0,2958.0,1057.0,288.0,8.32,6.93,18.05,50.52,5.33,3.983
268,2020-11-18T17:00:00,4,Trentino-Alto Adige,879,77,956,12805,13761,66,817,...,18.0,7692.0,1965.0,59.0,8.05,6.95,10.62,41.58,-2.53,5.966
269,2020-11-19T17:00:00,4,Trentino-Alto Adige,876,78,954,13236,14190,429,962,...,20.0,6135.0,1790.0,431.0,8.18,6.72,15.68,53.74,1.30,6.991
270,2020-11-20T17:00:00,4,Trentino-Alto Adige,885,76,961,13554,14515,325,970,...,15.0,6330.0,1663.0,318.0,7.91,6.62,15.32,58.33,-2.56,6.836


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
